In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, pipeline
import torch
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import classification_report
import pandas as pd
from nltk.corpus import stopwords

2024-04-30 18:52:48.701344: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-30 18:52:48.701390: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-30 18:52:48.704157: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-30 18:52:48.966768: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 18:52:50.277194: W tensorflow/compiler/

In [2]:
model_eval = ("albert_echr_model", "albert-base-v2")

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_eval[1])
stop_words = stopwords.words('english')

In [4]:
def encode(examples):
    return tokenizer( examples["text"],
                     truncation=True, 
                     padding=True)

In [5]:
test_df = pd.read_csv('../data/csv/test.csv')
test_df['text'] = test_df['text'].str.lower().str.split().apply(lambda x: [item for item in x if item not in stop_words]).apply(lambda x: " ".join(x)).replace('\d+', '', regex=True)
test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(encode, batched=True)

Map:   0%|          | 0/2998 [00:00<?, ? examples/s]

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
metrics = ["accuracy"] #, "f1", "precision", "recall"]
reports = dict()
for metric in metrics:
    print(f"Evaluating model {model_eval[0]} trained with metric {metric}")
    model = AutoModelForSequenceClassification.from_pretrained(f"../models/ECHR/{model_eval[0]}/removed_stopwords/{metric}/checkpoint-888", num_labels=2)
    model.to(device)
    
    pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=device)
    results = pipe(tokenizer.decode(case, clean_up_tokenization_spaces=True, skip_special_tokens=True) for case in test_dataset['input_ids'])
    
    predictions = list(result['label'] for result in results)
    
    report = classification_report(list('NON_VIOLATED' if outcome==0 else 'VIOLATED' for outcome in test_dataset['labels']), predictions)
    reports[metric] = report
    print("--------------------------------")
    

Evaluating model albert_echr_model trained with metric accuracy
--------------------------------


In [8]:
for metric, report in reports.items():
    print(f"Report for the model {model_eval[0]} \ntrained with {metric} metric:")
    print(report)
    print("-----------------------------------------------------")

Report for the model albert_echr_model 
trained with accuracy metric:
              precision    recall  f1-score   support

NON_VIOLATED       0.63      0.68      0.65      1024
    VIOLATED       0.83      0.79      0.81      1974

    accuracy                           0.75      2998
   macro avg       0.73      0.74      0.73      2998
weighted avg       0.76      0.75      0.76      2998

-----------------------------------------------------
